In [ ]:
# Kmeans 1

import numpy as np 
import pandas as pd
#visulaization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')
#data and model
from sklearn import datasets
from sklearn.cluster import KMeans
%matplotlib inline

iris=datasets.load_iris()
X=iris.data[:, :2]
y=iris.target

plt.scatter(X[:,0], X[:,1], c=y, cmap='gist_rainbow')
plt.xlabel('Spea1 Length', fontsize=18)
plt.ylabel('Sepal Width', fontsize=18)
plt.title('Iris Data Distribution', fontsize=20)
km = KMeans(n_clusters = 3,  random_state=21)
km.fit(X)
centers = km.cluster_centers_
print(centers)
new_labels = km.labels_
plt.scatter(X[:,0], X[:,1], c=new_labels, cmap='gist_rainbow')
plt.xlabel('Spea1 Length', fontsize=18)
plt.ylabel('Sepal Width', fontsize=18)
plt.title('Iris Data Distribution', fontsize=20)

new_labels = km.labels_
fig, axes = plt.subplots(1, 2, figsize=(16,8))
axes[0].scatter(X[:, 0], X[:, 1], c=y, cmap='gist_rainbow',
edgecolor='k', s=150)
axes[1].scatter(X[:, 0], X[:, 1], c=new_labels, cmap='jet',
edgecolor='k', s=150)
axes[0].set_xlabel('Sepal length', fontsize=18)
axes[0].set_ylabel('Sepal width', fontsize=18)
axes[1].set_xlabel('Sepal length', fontsize=18)
axes[1].set_ylabel('Sepal width', fontsize=18)
axes[0].tick_params(direction='in', length=10, width=5, colors='k', labelsize=20)
axes[1].tick_params(direction='in', length=10, width=5, colors='k', labelsize=20)
axes[0].set_title('Actual', fontsize=18)
axes[1].set_title('Predicted', fontsize=18)


# elbow method
x=iris.data[:, :4]
wcss=[]
for i in range(1 , 11):
  km=KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
  km.fit(x)
  wcss.append(km.inertia_)

#plotting to results to observe the elbow
plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()
kms=KMeans(n_clusters = 3, init='k-means++', max_iter=300, n_init=10, random_state=0)
ykmeans=kms.fit_predict(x)
print(ykmeans)

#plotting the centroids
plt.scatter(x[ykmeans == 0, 0], x[ykmeans ==0, 1], s=100, c='red', label='iris-setosa')
plt.scatter(x[ykmeans == 1, 0], x[ykmeans ==1, 1], s=100, c='blue', label='iris-versicolor')
plt.scatter(x[ykmeans == 2, 0], x[ykmeans ==2, 1], s=100, c='green', label='iris-virginica')
plt.scatter(kms.cluster_centers_[:, 0],kms.cluster_centers_[:, 1], s=100, c='black', label='Centroids')
plt.legend()

In [7]:
import numpy as np
import gym
import random

env = gym.make("FrozenLake-v1")

action_size = env.action_space.n
state_size = env.observation_space.n
# Create our Q table with state_size rows and action_size columns (64x4)
qtable = np.zeros((state_size, action_size))
print(qtable)

#Here, we'll specify the hyperparameters
total_episodes = 20000       # Total episodes
learning_rate = 0.7          # Learning rate
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005            # Exponential decay rate for exploration prob

#Now we implement the Q learning algorithm:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
            #print(exp_exp_tradeoff, "action", action)

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            #print("action random", action)
            
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)
    

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

#Use our Q-table to play FrozenLake ! 👾
#After 10 000 episodes, our Q-table can be used as a "cheatsheet" to play FrozenLake"
#By running this cell you can see our agent playing FrozenLake.

env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            if new_state == 15:
                print("We reached our Goal 🏆")
            else:
                print("We fell into a hole ☠️")
            
            # We print the number of step it took.
            print("Number of steps", step)
            
            break
        state = new_state
env.close()

#Now we implement the SARSA learning algorithm:
import random

import gym
import numpy as np
import matplotlib.pyplot as plt


env = gym.make('FrozenLake-v1')
print("Action space:", env.action_space)
env1 = env.unwrapped
print("State space:", env.env.nS)
print(env.env.P[0])
state_size = 16
action_space = env.action_space.n
alpha = 0.1
gamma = 1
state_action_vals = np.random.randn(state_size, action_space)
policy = np.zeros(state_size, dtype=int)
episodes = 20000
eps = 0.2
test_episodes = 50
test_every = 1000
test_episode = []
rewards = []
def select_action(state, eps):
    sample = np.random.uniform()
    if sample < eps:
        return env.action_space.sample()
    else:
        return state_action_vals[state].argmax()
    
for ep in range(episodes):
    state = env.reset()
    action = select_action(state, eps)
    done = False
    while not done:
        next_state, reward, done, _ = env.step(action)
        next_action = select_action(state, eps)

        action_value = state_action_vals[state, action]
        next_action_value = state_action_vals[next_state, next_action]
        delta = reward + gamma * next_action_value - action_value
        state_action_vals[state, action] += alpha * delta
        state, action = next_state, next_action
        
    if ep % test_every == 0:
        total_rewards = 0
        for _ in range(test_episodes):
            done = False
            state = env.reset()
            while not done:
                action = state_action_vals[state].argmax()
                state, reward, done, _ = env.step(action)
                total_rewards += reward
        rewards.append(total_rewards / test_episodes)
        test_episode.append(ep)
        

fig, ax = plt.subplots()
ax.plot(test_episode, rewards)
ax.set_title('Episodes vs average rewards')
ax.set_xlabel('Episode')
_ = ax.set_ylabel('Average reward')

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Score over time: 0.50075
[[7.08694501e-02 6.52301015e-02 1.10399235e-01 7.03162208e-02]
 [2.83529346e-02 3.26858767e-02 5.22959991e-02 7.06789594e-02]
 [2.20703046e-02 2.34452801e-02 5.05068333e-02 5.43064346e-02]
 [1.04108805e-02 4.06463113e-02 1.18206237e-03 5.47047155e-02]
 [3.21960606e-01 7.23880331e-02 7.16073413e-02 3.89906002e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.55963069e-04 2.63556983e-04 4.74803149e-02 3.02425827e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.79938112e-02 6.55224881e-03 7.40831670e-02 5.93851991e-01]
 [1.17231879e-01 8.18685603e-01 5.83189002e-02 3.16395827e-03]
 [4.11992630e-01 7.59974312e-04 1.00414065e-02 5.95768577e-03]
 [0.00000000e+00 0.00000000e+00 0.0000000

AttributeError: 'FrozenLakeEnv' object has no attribute 'nS'